# "Guess the Number"
Let's play game in which Computer finds out the number we have guessed.

### The game rules:

* Initially Computer gets the lower and the upper bounds of the range hidden number belongs to. 
* Computer predicts numbers until it guesses our number, which is always integer by the way. 
* After every wrong try Computer will be told if predicted number is more or if it's less 
than the one we made.

Thanks to game rules Computer has unlimited tries so it has no chance to loose. 

So the result of the game is a number of tries Computer will take.

### Game algorythm 
Game algoryth is placed in the only function – *guess_number()* from guess_number.py module.

In [33]:
from numpy import random

def guess_number(lo: int=1, hi: int=100, number: int=None, bnr_srch: bool=True) -> int:
    """
    Computer will try to guess hidden number.
    It has got unlimited tries. 
    The result is the number tries it will use to find out the number
    
    Args:
        lo (int, optional): lower bound of the range the number should belong to. 
        Defaults to 1.
        hi (int, optional): higher bound of the range the number should belong to. 
        Defaults to 100.
        number (int): gessed number.
        Defaults to None (randint(lo, hi+1) will give the number in this case)
        bnr_srch (bool, optional): if True binary search algorythm will be used
        to search the number, othervice Computer will use randint() function.
        Defaults to True.

    Returns:
        int: number of tries Computer have used to guess the number.
    """

    # Check arguments values:
    # lo, hi – int; number – None or int
    if not ((number == None or type(number) == int) and type(lo) == int and type(hi) == int):
        raise ValueError("number, lo, hi values should be integer")
    if lo > hi:
        raise ValueError("lo can't be more than hi")
    if not number:
        number = random.randint(lo, hi+1)
    # number value should be in range [lo, hi]
    if not lo <= number <= hi:
        raise ValueError("number should be in range from lo to hi")
    
    # Now let Computer guess the number
    count = 1
    while True:
        if bnr_srch: # 
            guess = (lo+hi) // 2
        else:
            guess = random.randint(lo, hi+1)
            
        if guess == number:
            return count
        elif guess < number:
            lo = guess+1
        else:
            hi = guess-1
        count += 1

In [34]:
from guess_number import guess_number
from numpy import mean, random
from math import log2, ceil

def average_int_result(func, repeat: int=1000, print_chart: bool=True, bar_len: int=35):
    """
    Wrapper for collecting and processing function multiple run results.
    This decorator is useless if the wrapped function returns the same result 
    every time if arguments are the same. 
    Otherwise, use this decorator to calculate the average return value 
    of multiple function runs. 
    In addition, decorator can be used to output a simple diagram 
    to visualize the frequency of values returned by the wrapped function.

    Args:
        func (_type_): wrapped function
        repeat (int, optional): number of function runs. Defaults to 1000.
        print_chart (bool, optional): Defaults to True.
        bar_len (int, optional): Lenght of the longest chart bar. Defaults to 35.
    """
    def dec_func(*args, **kwargs):
        results = [func(*args, **kwargs) for i in range(repeat)]
        avrg_res = round(mean(results))
        
        if print_chart:
            # Printing a simple chart
            min_res = min(results) # lower bound of chart value axis
            max_res = max(results) # upper bound of chart value axis
            # Creating a dictionary:
            # keys are results returned by func();
            # values are numbers of times each result was returned.
            results = {i: results.count(i) 
                        for i in range(min_res, max_res+1)}
            max_val = max(results.values()) # count of the most frequent result
            
            for result in results.items():
                if result[1]: # Only non-zero bars are printed
                    # Mark the average value bar with different symbol
                    chart_ch = '=' if result[0] == avrg_res else '-'
                    indent = ' '*(len(str(max_res)) - len(str(result[0])))
                    chart_bar = indent + chart_ch * ceil(result[1]/max_val * bar_len)
                    print(result[0], chart_bar, f"{result[1]} times")
                
        return avrg_res
    return dec_func

In [35]:
guess_number_chart = average_int_result(guess_number, print_chart=True)

random.seed(1)

print(f"Average number of tries is {guess_number_chart(bnr_srch=False)}")
print("\nNow using binary search algorythm:")
print(f"Average number of tries is {guess_number_chart(bnr_srch=True)}")

guess_number = average_int_result(guess_number, print_chart=False)
for hi in [2**i for i in range(10,21)]:
    print(f"\n[1; 2**{int(log2(hi))}]")
    print(f"Average number of tries is {guess_number(1,hi)}")

1  --- 9 times
2  ----- 19 times
3  -------- 34 times
4  ---------------- 68 times
5  --------------------------- 115 times
6  -------------------------------- 139 times
7  =================================== 154 times
8  --------------------------------- 142 times
9  ------------------------ 105 times
10 --------------------- 89 times
11 -------------- 60 times
12 ------- 28 times
13 ----- 20 times
14 --- 13 times
15 - 1 times
16 - 4 times
Average number of tries is 7

Now using binary search algorythm:
1 - 10 times
2 --- 28 times
3 ---- 38 times
4 ------- 74 times
5 ------------- 138 times
6 ============================== 326 times
7 ----------------------------------- 386 times
Average number of tries is 6

[1; 2**10]
Average number of tries is 9

[1; 2**11]
Average number of tries is 10

[1; 2**12]
Average number of tries is 11

[1; 2**13]
Average number of tries is 12

[1; 2**14]
Average number of tries is 13

[1; 2**15]
Average number of tries is 14

[1; 2**16]
Average number of 